# 03-AdditionalLayers  
Can we extract additional features from the image? 

In [1]:
%load_ext autoreload
%autoreload 2

import gi_tract_seg as gt
from gi_tract_seg.models.module import GITractSegmentatonLitModule
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import seaborn as sns

In [144]:
def plot_sus_pairs(case_day_data, case, day, idx, save):
    data = datamodule.train_dataset[case_day_data.index.values[idx]]
    mask = data['mask'].numpy().transpose(1,2,0)
    image = np.tile(data['image'].numpy().transpose(1,2,0).astype(np.float32), 3)
    data = datamodule.train_dataset[case_day_data.index.values[idx+1]]
    mask_next = data['mask'].numpy().transpose(1,2,0)
    image_next = np.tile(data['image'].numpy().transpose(1,2,0).astype(np.float32), 3)
    image_dif = image[..., 0] - image_next[..., 0]
    image = cv2.addWeighted(
        src1=image, 
        alpha=0.999, 
        src2=mask, 
        beta=0.35, 
        gamma=0
    )
    image_next = cv2.addWeighted(
        src1=image_next, 
        alpha=0.999, 
        src2=mask_next, 
        beta=0.35, 
        gamma=0
    )    
    f, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image)
    ax[1].imshow(image_next)
    ax[2].imshow(image_dif, cmap='coolwarm')
    plt.tick_params()
    if save:
        f.savefig(f'images/sus_annotation//{case}_{day}_{idx}.jpg')
    plt.close()

In [2]:
datamodule = gt.data.datamodule.GITractDataModule(
    '../data/processed/train_df_agg.parquet', 
    '../data/raw/train/', 
    '../data/processed/masks/', 
    val_fold=0,
    batch_size=128,
    num_workers=2,
    shuffle_train=False
)
datamodule.prepare_data()
datamodule.setup()

In [24]:
dice_meter = gt.models.metrics.DiceMeter('multilabel')

In [106]:
train_df = datamodule.train_data.reset_index(drop=False)
train_df.head()

,index,id,case,day,slice,image_path,height,width,classes,segmentation,labels,empty,mask_path,segmentation_sizes,fold
0,0,case123_day20_slice_0001,123,20,1,case123/case123_day20/scans/slice_0001_266_266...,266,266,"[large_bowel, small_bowel, stomach]","[None, None, None]","[0, 0, 0]",1,case123/case123_day20/scans/slice_0001_266_266...,"[0.0, 0.0, 0.0]",3
1,1,case123_day20_slice_0002,123,20,2,case123/case123_day20/scans/slice_0002_266_266...,266,266,"[large_bowel, small_bowel, stomach]","[None, None, None]","[0, 0, 0]",1,case123/case123_day20/scans/slice_0002_266_266...,"[0.0, 0.0, 0.0]",3
2,2,case123_day20_slice_0003,123,20,3,case123/case123_day20/scans/slice_0003_266_266...,266,266,"[large_bowel, small_bowel, stomach]","[None, None, None]","[0, 0, 0]",1,case123/case123_day20/scans/slice_0003_266_266...,"[0.0, 0.0, 0.0]",3
3,3,case123_day20_slice_0004,123,20,4,case123/case123_day20/scans/slice_0004_266_266...,266,266,"[large_bowel, small_bowel, stomach]","[None, None, None]","[0, 0, 0]",1,case123/case123_day20/scans/slice_0004_266_266...,"[0.0, 0.0, 0.0]",3
4,4,case123_day20_slice_0005,123,20,5,case123/case123_day20/scans/slice_0005_266_266...,266,266,"[large_bowel, small_bowel, stomach]","[None, None, None]","[0, 0, 0]",1,case123/case123_day20/scans/slice_0005_266_266...,"[0.0, 0.0, 0.0]",3


In [140]:
unique_case_days = train_df.apply(lambda x: str(x['case'])+"_"+str(x['day']),axis=1).unique()

In [148]:
for case_day_idx in range(len(unique_case_days)):
    case = int(unique_case_days[case_day_idx].split('_')[0])
    day = int(unique_case_days[case_day_idx].split('_')[1])
    case_day_data = train_df.loc[(train_df['case']==case) & (train_df['day']==day)]
    dice_scores = []
    for idx in tqdm(range(case_day_data.shape[0]-1)):
        data = datamodule.train_dataset[case_day_data.index.values[idx]]
        mask = data['mask']
        data = datamodule.train_dataset[case_day_data.index.values[idx+1]]
        mask_next = data['mask']
        dice_scores.append(dice_meter.score_fn(mask, mask_next))
        mask = mask_next

    masks_shift = np.nanmean(np.stack(dice_scores), 1)
    sus_annotation_idx = np.where(masks_shift<=0.6)[0]
    for sus_an_idx in sus_annotation_idx:
        plot_sus_pairs(case_day_data, case, day, sus_an_idx, True)

  0%|          | 0/143 [00:00<?, ?it/s]

<ipython-input-148-7db04d123dcc>:14: RuntimeWarning: Mean of empty slice
  masks_shift = np.nanmean(np.stack(dice_scores), 1)


  0%|          | 0/143 [00:00<?, ?it/s]

<ipython-input-148-7db04d123dcc>:14: RuntimeWarning: Mean of empty slice
  masks_shift = np.nanmean(np.stack(dice_scores), 1)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


  0%|          | 0/143 [00:00<?, ?it/s]

...

<ipython-input-148-7db04d123dcc>:14: RuntimeWarning: Mean of empty slice
  masks_shift = np.nanmean(np.stack(dice_scores), 1)


  0%|          | 0/143 [00:00<?, ?it/s]

<ipython-input-148-7db04d123dcc>:14: RuntimeWarning: Mean of empty slice
  masks_shift = np.nanmean(np.stack(dice_scores), 1)


  0%|          | 0/143 [00:00<?, ?it/s]

<ipython-input-148-7db04d123dcc>:14: RuntimeWarning: Mean of empty slice
  masks_shift = np.nanmean(np.stack(dice_scores), 1)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


what if we simply drop incosistent annotations